### Introduction

In this project, you will be continuing your work with the *Escherichia coli* core model and investigate how the growth phenotype of the model changes when varying one or multiple exchange fluxes. Additionally, you will be conducting a gene essentiality analysis on the model where the deletion of a gene can be simulated by simply constraining the lower and upper flux bounds of its associated reactions to 0. Gene deletion analysis is a common application of genome-scale metabolic models and allow us to assess the effects of a gene deletion on the metabolic phenotype of an organism before initiating expensive and time-consuming experiments in the lab. I recommend taking a look at the `cobrapy` documentation before starting with the project, particularly the section on deletion analysis found [here](https://cobrapy.readthedocs.io/en/latest/deletions.html).


### 1.1 Phenotype phase plane analysis

(i) Using the `cobrapy` function `production_envelope` (see [documentation](https://cobrapy.readthedocs.io/en/latest/phenotype_phase_plane.html)), calculate the growth rate as a function of glucose uptake flux when oxygen is unlimited for the *E. coli* core model. Plot growth vs. glucose uptake flux from 0 to -25 mmol gDW<sup>-1</sup> h<sup>-1</sup>.

In [ ]:
import cobra
import matplotlib.pyplot as plt

# Load the E. coli core model from XML file
model = cobra.io.read_sbml_model('/Users/edgartandberg/Documents/Personlig/Vs_code/Systembiologi_v2024/Systembio_prosjekt_4/ecoli_core_model.xml')

model.objective = 'BIOMASS_Ecoli_core_w_GAM'

#Define the range of glucose uptake flux
glucose_range = range(0, -26, -1)

# Calculate the growth rate for each glucose uptake flux
growth_rates = []
for glucose_flux in glucose_range:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = glucose_flux
    solution = model.optimize()
    growth_rate = solution.objective_value
    growth_rates.append(growth_rate)

#Plot growth rate vs. glucose uptake flux
plt.plot(glucose_range, growth_rates)
plt.xlabel('Glucose Uptake Flux (mmol gDW^-1 h^-1)')
plt.ylabel('Growth Rate (1/h)')
plt.title('Growth Rate vs. Glucose Uptake Flux')
plt.show()




(ii) Same as in (i), but now limit the oxygen uptake flux to -10 mmol gDW<sup>-1</sup> h<sup>-1</sup> . Compare and explain the differences with the results from (i).

In [ ]:
# Set the oxygen uptake flux to -10 mmol gDW^-1 h^-1
model.reactions.get_by_id('EX_o2_e').lower_bound = -10

# Calculate the growth rate for each glucose uptake flux
growth_rates = []
for glucose_flux in glucose_range:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = glucose_flux
    solution = model.optimize()
    growth_rate = solution.objective_value
    growth_rates.append(growth_rate)

# Plot growth rate vs. glucose uptake flux
plt.plot(glucose_range, growth_rates)
plt.xlabel('Glucose Uptake Flux (mmol gDW^-1 h^-1)')
plt.ylabel('Growth Rate (1/h)')
plt.title('Growth Rate vs. Glucose Uptake Flux (Oxygen limited)')
plt.show()


(iii) Same as in (i) and (ii), but with succinate as the only carbon source. Compare and explain the differences with the results from (i) and (ii). Use the calculated growth rates from Table 1 in Project 3 to argue for your results.

In [ ]:
# Set the carbon source to succinate
model.reactions.get_by_id('EX_succ_e').lower_bound = -10

# Calculate the growth rate for each succinate uptake flux
growth_rates = []
for succinate_flux in glucose_range:
    model.reactions.get_by_id('EX_succ_e').lower_bound = succinate_flux
    solution = model.optimize()
    growth_rate = solution.objective_value
    growth_rates.append(growth_rate)

# Plot growth rate vs. succinate uptake flux
plt.plot(glucose_range, growth_rates)
plt.xlabel('Succinate Uptake Flux (mmol gDW^-1 h^-1)')
plt.ylabel('Growth Rate (1/h)')
plt.title('Growth Rate vs. Succinate Uptake Flux')
plt.show()


Commonly, we classify the linear segments of the production envelopes by their *shadow prices*, which are defined as

$
\begin{align}
 \pi_i = -\frac{\partial Z}{\partial b_i}.
\end{align}
$

The shadow prices $\pi_i$ defines the sensitivity of the objective function $Z$ to changes in the availability of a given metabolite $i$. $b_i$ defines the violation of the mass balance constraint of metabolite $i$ and is equivalent to an uptake reaction. In other words, it quantitatively describes how the objective function value would change if we could import more of an exchangeable metabolite. A negative $\pi_i$ indicates that increasing the availability of a metabolite $i$ will increase the optimal objective value, while a positive $\pi_i$ indicates that the optimal objective value will decrease.

While the production envelopes plotted above are interesting in their own right, it is often more informative to investigate the growth phenotype when adjusting two exchange fluxes simultaneously. This is called phenotype phase plane analysis (PhPP). Rather than forming linear segments, PhPP results in the formation of two-dimensional planes which are defined by their metabolic phenotype and shadow prices of the two variable exchange metabolites.

(iv) Start by plotting a PhPP and associated shadow prices for glucose and oxygen using the *E. coli* core model and the `PhPP` and associated plotting functions found in the `tbtFunctions` module. Use a `flux_range` evenly distributed between 0 and -20 with 50 data points.

In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)

In [ ]:
import importlib.util
from matplotlib import pyplot as plt

# Specify the file path
file_path = '/Users/edgartandberg/Documents/Personlig/Vs_code/Systembiologi_v2024/Systembio_prosjekt_4/tbtFunctions(1).py'

# Import the module
spec = importlib.util.spec_from_file_location('tbtFunctions', file_path)
tbtFunctions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(tbtFunctions)

# Define the flux range
flux_range = range(0, -21, -1)

# Calculate the PhPP and shadow prices
tbtFunctions.PhPP(model, 'EX_glc__D_e', 'EX_o2_e', flux_range)

# Plot the PhPP and shadow prices
plt.xlabel('Glucose Uptake Flux (mmol gDW^-1 h^-1)')
plt.ylabel('Oxygen Uptake Flux (mmol gDW^-1 h^-1)')
plt.title('Phenotype Phase Plane (Glucose vs. Oxygen)')
plt.show()


We can observe that the shadow prices $\pi_i$ are constant in each of the phase planes. Consequently, we can determine the properties of the different phases by calculating

$
\begin{align}
 \alpha = -\frac{\pi_x}{\pi_y},
\end{align}
$

the ratio between the shadow prices associated with the two exchange fluxes $x$ and $y$ (here, glucose and oxygen). We can then define four types of phases in PhPP:
1. If $\alpha < 0$, the objective function is limited by both exchange fluxes (*dual substrate limitation*).
2. If $\alpha > 0$, the region is called a *futile phase*: one of the exchange substrates will have a positive shadow price. Characterized by wasteful metabolic operations (substrate consumed does not improve $Z$).
3. If either $\pi_x = 0$ or $\pi_y = 0$, the region is described by *single substrate limitation*.
4. *Infeasible phases* due to a combination of stoichiometric limitations and/or substrate availability (no growth).

(v) How many unique phases does this PhPP have? Classify these according to the four types defined above.

(vi) Considering the reasonable assumption of growth optimality, where do you think *E. coli* strains will be located on this PhPP in vivo?

### 1.2 Gene deletion analysis

(i) Conduct a single gene deletion study for all the genes in the *E. coli* core model. Use the default defined nutrient environment (i.e. aerobic glucose).

(a) By making a scatter plot of the relative growth rates (mutant growth / wild type growth) for all gene deletions, define what you consider to be an essential gene.

In [ ]:
import cobra
import matplotlib.pyplot as plt

# Load the E. coli core model
model = cobra.io.read_sbml_model('ecoli_core_model.xml')

# Perform single gene deletion study
deletion_results = cobra.flux_analysis.single_gene_deletion(model)

# Get the growth rates for each gene deletion
growth_rates = deletion_results.growth_rate

# Calculate the relative growth rates
relative_growth_rates = growth_rates / deletion_results.wildtype_growth_rate

# Plot the scatter plot of relative growth rates
plt.scatter(range(len(relative_growth_rates)), relative_growth_rates)
plt.xlabel('Gene Index')
plt.ylabel('Relative Growth Rate')
plt.title('Relative Growth Rates for Single Gene Deletions')
plt.show()


(b) Identify the name and function of all the essential genes you found. While checking which reactions these genes are associateed with in the model is useful, it can also be an idea to check biochemical databases for informations such as [EcoCyc](https://ecocyc.org/).

In [ ]:
# Get the essential genes
essential_genes = deletion_results[deletion_results['growth_rate'] == 0]

# Print the name and function of each essential gene
for gene_id in essential_genes.index:
    gene = model.genes.get_by_id(gene_id)
    print(f"Gene ID: {gene_id}")
    print(f"Name: {gene.name}")
    print(f"Function: {gene.annotation['function']}")
    print()


(c) Give plausible explanations for why they are essential.

(ii) Change the single carbon source from glucose to acetate and redo the analysis in (a) (no need to go into detail on the function of each gene, comment on the general trend). Also, identify the essential genes that are common to both nutrient environments, and those that are specific to acetate. Explain why this is so.

In [ ]:
# Set the carbon source to acetate
model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
model.reactions.get_by_id('EX_ac_e').lower_bound = -10

# Perform single gene deletion study
deletion_results_acetate = cobra.flux_analysis.single_gene_deletion(model)

# Get the growth rates for each gene deletion
growth_rates_acetate = deletion_results_acetate.growth_rate

# Calculate the relative growth rates
relative_growth_rates_acetate = growth_rates_acetate / deletion_results_acetate.wildtype_growth_rate

# Plot the scatter plot of relative growth rates for acetate
plt.scatter(range(len(relative_growth_rates_acetate)), relative_growth_rates_acetate)
plt.xlabel('Gene Index')
plt.ylabel('Relative Growth Rate')
plt.title('Relative Growth Rates for Single Gene Deletions (Acetate)')
plt.show()


(iii) Conduct a complete double-gene deletion study of the model in the original nutrient environment.

(a) What is the fraction of essential double-gene knockouts?

In [ ]:
# Calculate the fraction of essential double-gene knockouts
fraction_essential = len(double_gene_knockouts[double_gene_knockouts['growth_rate'] == 0]) / len(double_gene_knockouts)

# Print the fraction of essential double-gene knockouts
print(f"The fraction of essential double-gene knockouts is: {fraction_essential}")


(b)  Which genes are involved in the largest number of essential double-knockout combinations? Any idea why?

(iv) Same analysis as in (iii), but this time with acetate as the single carbon source.

In [ ]:
# Set the carbon source to acetate
model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
model.reactions.get_by_id('EX_ac_e').lower_bound = -10

# Perform single gene deletion study
deletion_results_acetate = cobra.flux_analysis.single_gene_deletion(model)

# Get the growth rates for each gene deletion
growth_rates_acetate = deletion_results_acetate.growth_rate

# Calculate the relative growth rates
relative_growth_rates_acetate = growth_rates_acetate / deletion_results_acetate.wildtype_growth_rate

# Plot the scatter plot of relative growth rates for acetate
plt.scatter(range(len(relative_growth_rates_acetate)), relative_growth_rates_acetate)
plt.xlabel('Gene Index')
plt.ylabel('Relative Growth Rate')
plt.title('Relative Growth Rates for Single Gene Deletions (Acetate)')
plt.show()


(v) Use the results from (i) and (iii) to identify only the synthetic lethal gene combinations. How many are there?

In [ ]:
# Get the essential genes from the single gene deletion study
essential_genes = deletion_results[deletion_results['growth_rate'] == 0]

# Get the gene pairs where both genes are essential
synthetic_lethal_pairs = []
for i in range(len(essential_genes)):
    for j in range(i+1, len(essential_genes)):
        gene1 = essential_genes.index[i]
        gene2 = essential_genes.index[j]
        synthetic_lethal_pairs.append((gene1, gene2))

# Count the number of synthetic lethal gene combinations
num_synthetic_lethal_pairs = len(synthetic_lethal_pairs)

# Print the number of synthetic lethal gene combinations
print(f"The number of synthetic lethal gene combinations is: {num_synthetic_lethal_pairs}")


(vi) Select a partial essential gene from (a) (i.e., a gene which when deleted reduces the relative growth rate of the model, but not below the chosen essentiality threshold). Visualize its optimal flux distribution in Escher and compare it to the wild type flux distribution. Discuss and explain the observed differences in flux phenotypes.

In [ ]:
import cobra
import escher
from escher import Builder

# Load the E. coli core model
model = cobra.io.read_sbml_model('ecoli_core_model.xml')

# Select a partial essential gene
partial_essential_gene = 'gene_name'

# Delete the partial essential gene from the model
model.genes.get_by_id(partial_essential_gene).knock_out()

# Optimize the model to obtain the optimal flux distribution
solution = model.optimize()

# Get the optimal flux distribution
flux_distribution = solution.fluxes

# Load the Escher map
builder = Builder(map_name='e_coli_core.Core metabolism')

# Set the fluxes in the Escher map
builder.reaction_data = flux_distribution.to_dict()

# Display the Escher map
builder.display_in_notebook()
